In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
dataset, info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [4]:
train = train_dataset.take(4000)
test = test_dataset.take(1000)


In [5]:
# to shuffle the data...
BUFFER_SIZE = 4000 # We will put all the data into this big buffer, and sample randomly from the buffer
BATCH_SIZE = 128 # We will read 128 reviews at a time

train = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)

In [6]:
train = train.prefetch(BUFFER_SIZE)
test = test.prefetch(BUFFER_SIZE)

In [7]:
VOCAB_SIZE = 1000 #assuming our vocabulary is just 1000 words

encoder = layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train.map(lambda text, label: text))
# we just  encode teh text not the labels

In [8]:
# here are the first 20 words in our 1000-word vocabulary
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'i', 'it',
       'this', 'that', 'br', 'was', 'as', 'with', 'for', 'but', 'movie'],
      dtype='<U14')

In [9]:
example, label = list(train.take(1))[0] # that's one batch

len(example)

128

In [10]:
example[0].numpy()

b"I've seen the original non-dubbed German version and I was surprised how bad this movie actually is. Thinking I had seen my share of bad movies like Ghoulies 2, Rabid Grannies, Zombie Lake and such, nothing could've prepared me for this! It really was a pain to sit through this flick, as there's no plot, no good acting and even the special effects aren't convincing, especially the so-called zombies, wearing nothing more than white make-up and their old clothes, so their good set wouldn't be ruined by ketchup and marmalade stains. <br /><br />If you really want to waste 90 minutes of your life, then watch it, for all the others, don't do it, because you WILL regret it!"

In [11]:
encoded_example = encoder(example[:1]).numpy()
encoded_example

array([[211, 111,   2, 203,   1,   1, 299,   3,   9,  14, 715,  87,  88,
         11,  19, 160,   7, 510,   9,  63, 111,  54,   1,   5,  88,  99,
         39,   1, 267,   1,   1, 881,   1,   3, 139, 162,   1,   1,  70,
         17,  11,  10,  62,  14,   4,   1,   6, 842, 135,  11, 556,  15,
        222,  58, 106,  58,  50, 112,   3,  55,   2, 316, 309, 595,   1,
        260,   2,   1,   1,   1, 162,  51,  71, 493,   1,   3,  66, 166,
          1,  36,  66,  50, 280, 584,  28,   1,  33,   1,   3,   1,   1,
         13,  13,  43,  25,  62, 177,   6, 462,   1, 232,   5, 133, 115,
         91, 104,  10,  17,  32,   2, 402,  89,  82,  10,  81,  25,  83,
          1,  10]], dtype=int64)

In [12]:
model = tf.keras.Sequential([encoder, #the encoder
                             tf.keras.layers.Embedding(
                             input_dim=len(encoder.get_vocabulary()),
                             output_dim=64,
                             # Use masking to handle the variable sequence lengths
                             mask_zero=True),
                             tf.keras.layers.Bidirectional(layers.LSTM(64)),
                             # making LSTM Bidirectional
                             tf.keras.layers.Dense(32, activation='relu'), 
                             # FC layer for the classification part
                             tf.keras.layers.Dense(1) # final FC layer
                             ])

In [13]:
sample_text = ('The movie was cool. The animation and the graphics were out of this world. I would reccomend this movie' )

predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00707749]


In [14]:
# we will use binary cross entropy again because this isa  binary classification
# problem because this is a binary classification task (positive or negative)
# we also did not apply a sigmoid activation fucntion at the last FC layer,
# so we specify that they are calculating the cross entropy from logits

model.compile(

loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#adam optimizer is more efficient (not always the most accurate though)
optimizer = tf.keras.optimizers.Adam(1e-4),
metrics=['accuracy']
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          64000     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 134,209
Trainable params: 134,209
Non-trai

In [16]:
H2 = model.fit(train, epochs=25, validation_data=test)

Epoch 1/25
32/32 [==============================] - 361s 11s/step - loss: 0.6930 - accuracy: 0.4967 - val_loss: 0.6928 - val_accuracy: 0.5030
Epoch 2/25
32/32 [==============================] - 382s 12s/step - loss: 0.6919 - accuracy: 0.4967 - val_loss: 0.6922 - val_accuracy: 0.5030
Epoch 3/25
32/32 [==============================] - 433s 14s/step - loss: 0.6904 - accuracy: 0.4967 - val_loss: 0.6910 - val_accuracy: 0.5030
Epoch 4/25
32/32 [==============================] - 420s 13s/step - loss: 0.6875 - accuracy: 0.4967 - val_loss: 0.6881 - val_accuracy: 0.5030
Epoch 5/25
32/32 [==============================] - 454s 14s/step - loss: 0.6792 - accuracy: 0.4967 - val_loss: 0.6771 - val_accuracy: 0.5100
Epoch 6/25
32/32 [==============================] - 458s 14s/step - loss: 0.6281 - accuracy: 0.6075 - val_loss: 0.6137 - val_accuracy: 0.6380
Epoch 7/25
32/32 [==============================] - 451s 14s/step - loss: 0.5633 - accuracy: 0.7060 - val_loss: 0.5708 - val_accuracy: 0.7440
Epoch 